In [1]:
# -*- coding: utf-8 -*-

from douzero.env.game import GameEnv
from douzero.evaluation.deep_agent import DeepAgent

EnvCard2RealCard = {3: '3', 4: '4', 5: '5', 6: '6', 7: '7',
                    8: '8', 9: '9', 10: 'T', 11: 'J', 12: 'Q',
                    13: 'K', 14: 'A', 17: '2', 20: 'X', 30: 'D'}

RealCard2EnvCard = {'3': 3, '4': 4, '5': 5, '6': 6, '7': 7,
                    '8': 8, '9': 9, 'T': 10, 'J': 11, 'Q': 12,
                    'K': 13, 'A': 14, '2': 17, 'X': 20, 'D': 30}

AllEnvCard = [3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7,
              8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12,
              12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 17, 17, 17, 17, 20, 30]

class ConsoleDouDiZhu:
    def __init__(self):
        # 模型路径
        self.card_play_model_path_dict = {
            'landlord': "baselines/douzero_WP/landlord.ckpt",
            'landlord_up': "baselines/douzero_WP/landlord_up.ckpt",
            'landlord_down': "baselines/douzero_WP/landlord_down.ckpt"
        }
        self.init_display()

    def init_display(self):
        print("="*50)
        print("斗地主AI辅助工具 - 命令行版")
        print("="*50)
        print("\n游戏说明:")
        print("1. 输入手牌时使用字母表示: 3-9直接输入数字, 10=T, J=J, Q=Q, K=K, A=A, 2=2, 小王=X, 大王=D")
        print("2. 出牌时直接输入牌的组合，如 '345' 或 'TT'，或留空表示'不出'")
        print("3. 首回合不能不出，连续两个玩家不出后必须出牌\n")

    def run(self):
        self.user_hand_cards_real = input("请输入你的手牌(如: 3456789TJQKA2XD): ").upper()
        self.three_landlord_cards_real = input("请输入三张底牌(如: 2XD): ").upper()
        
        print("\n请选择你的角色:")
        print("1. 地主上家")
        print("2. 地主")
        print("3. 地主下家")
        self.user_position_code = int(input("请输入数字(1-3): ")) - 1
        self.user_position = ['landlord_up', 'landlord', 'landlord_down'][self.user_position_code]
        
        if not self.initialize_card_library():
            print("牌库校验失败，请检查输入！")
            return
        
        self.process_after_input()

    def initialize_card_library(self):
        """初始化牌库，记录剩余牌数"""
        # 总牌库
        card_counts = {'3': 4, '4': 4, '5': 4, '6': 4, '7': 4, '8': 4, '9': 4, 'T': 4,
                      'J': 4, 'Q': 4, 'K': 4, 'A': 4, '2': 4, 'X': 1, 'D': 1}

        # 扣除玩家手牌
        for card in self.user_hand_cards_real:
            if card not in card_counts:
                print(f"错误: 无效的牌 '{card}'")
                return False
            card_counts[card] -= 1
            if card_counts[card] < 0:
                print(f"错误: 牌 '{card}' 的数量不足！")
                return False

        # 只有当玩家是地主时才扣除底牌
        if self.user_position == "landlord":
            for card in self.three_landlord_cards_real:
                if card not in card_counts:
                    print(f"错误: 无效的牌 '{card}'")
                    return False
                card_counts[card] -= 1
                if card_counts[card] < 0:
                    print(f"错误: 牌 '{card}' 的数量不足！")
                    return False

        self.card_library = card_counts
        self.update_card_library_display()
        return True

    def update_card_library_display(self):
        """更新牌库显示"""
        print("\n当前牌库剩余:")
        for card, count in sorted(self.card_library.items()):
            if count > 0:
                print(f"{card}: {count}张", end=" | ")
        print()

    def validate_and_update_card_library(self, played_cards):
        """验证并更新牌库"""
        if not played_cards:  # 不出牌
            return True

        temp_lib = self.card_library.copy()
        for card in played_cards:
            if card not in temp_lib:
                print(f"错误: 牌 '{card}' 不存在！")
                return False
            if temp_lib[card] <= 0:
                print(f"错误: 牌 '{card}' 已用完！")
                return False
            temp_lib[card] -= 1

        self.card_library = temp_lib
        self.update_card_library_display()
        return True

    def process_after_input(self):
        # 整副牌减去玩家手上的牌，就是其他人的手牌
        self.user_hand_cards_env = [RealCard2EnvCard[c] for c in list(self.user_hand_cards_real)]
        self.three_landlord_cards_env = [RealCard2EnvCard[c] for c in list(self.three_landlord_cards_real)]
        
        # 如果玩家是地主，将底牌加入手牌
        if self.user_position == "landlord":
            self.user_hand_cards_env.extend(self.three_landlord_cards_env)
            self.user_hand_cards_real += self.three_landlord_cards_real
        
        self.other_hand_cards = []
        for i in set(AllEnvCard):
            self.other_hand_cards.extend([i] * (AllEnvCard.count(i) - self.user_hand_cards_env.count(i)))

        self.card_play_data_list = {
            'three_landlord_cards': self.three_landlord_cards_env,
            ['landlord_up', 'landlord', 'landlord_down'][(self.user_position_code + 0) % 3]: self.user_hand_cards_env,
            ['landlord_up', 'landlord', 'landlord_down'][(self.user_position_code + 1) % 3]:
                self.other_hand_cards[0:17] if (self.user_position_code + 1) % 3 != 1 else self.other_hand_cards[17:],
            ['landlord_up', 'landlord', 'landlord_down'][(self.user_position_code + 2) % 3]:
                self.other_hand_cards[0:17] if (self.user_position_code + 1) % 3 == 1 else self.other_hand_cards[17:]
        }

        # 校验手牌数量
        if len(self.card_play_data_list["three_landlord_cards"]) != 3:
            print("错误: 底牌必须是3张！")
            return

        # 修改手牌数量校验逻辑
        expected_landlord_up = 17
        expected_landlord_down = 17
        expected_landlord = 20 if self.user_position == "landlord" else 17
        
        if len(self.card_play_data_list["landlord_up"]) != expected_landlord_up or \
                len(self.card_play_data_list["landlord_down"]) != expected_landlord_down or \
                len(self.card_play_data_list["landlord"]) != expected_landlord:
            print(f"错误: 初始手牌数目有误 (地主上家:{len(self.card_play_data_list['landlord_up'])}张, " +
                  f"地主下家:{len(self.card_play_data_list['landlord_down'])}张, " +
                  f"地主:{len(self.card_play_data_list['landlord'])}张)")
            return

        # 得到出牌顺序
        self.play_order = 0 if self.user_position == "landlord" else 1 if self.user_position == "landlord_up" else 2

        # 创建一个代表玩家的AI
        ai_players = [0, 0]
        ai_players[0] = self.user_position
        ai_players[1] = DeepAgent(self.user_position, self.card_play_model_path_dict[self.user_position])

        self.env = GameEnv(ai_players)
        self.start()

    def start(self):
        self.env.card_play_init(self.card_play_data_list)
        print("\n游戏开始！")
        self.consecutive_passes = 0  # 初始化计数器
        self.first_round = True  # 初始化首回合标志
        
        while not self.env.game_over:
            print("\n" + "="*30)
            print(f"当前手牌: {''.join([EnvCard2RealCard[c] for c in sorted(self.env.info_sets[self.user_position].player_hand_cards)])}")
            
            if self.play_order == 0:
                # 玩家出牌 - 自动获取AI建议
                print("\n[AI正在思考...]")
                action_message = self.env.step(self.user_position)
                action_str = action_message["action"] if action_message["action"] else "不出"
                
                print(f"\nAI建议: {action_str} (胜率: {action_message['win_rate']})")
                if action_str == "不出":
                    self.consecutive_passes += 1
                else:
                    self.consecutive_passes = 0
                self.first_round = False
                self.play_order = 1
                print(f"[玩家出牌]: {action_str}")

            elif self.play_order == 1:
                # 下家出牌 - 手动输入
                must_play = self.consecutive_passes >= 1 or self.first_round
                prompt = "请输入下家出的牌(如:345或留空表示不出)"
                if must_play:
                    prompt += " (必须出牌): "
                else:
                    prompt += ": "
                    
                while True:
                    text = input(prompt).upper()
                    if not text:  # 不出
                        if must_play:
                            print("错误: 首回合不能不出或连续两个玩家不出后必须出牌！")
                            continue
                        else:
                            self.consecutive_passes += 1
                            self.other_played_cards_real = ""
                            break
                    # 验证牌型
                    if self.validate_card_pattern(text) and self.validate_and_update_card_library(text):
                        self.other_played_cards_real = text
                        self.consecutive_passes = 0
                        if self.first_round:
                            self.first_round = False
                        break
                    else:
                        print("错误: 牌型不符合规则或牌库中无此牌！")

                self.other_played_cards_env = [RealCard2EnvCard[c] for c in list(self.other_played_cards_real)]
                self.env.step(self.user_position, self.other_played_cards_env)
                self.play_order = 2
                print(f"[下家出牌]: {self.other_played_cards_real if self.other_played_cards_real else '不出'}")

            elif self.play_order == 2:
                # 上家出牌 - 手动输入
                must_play = self.consecutive_passes >= 1 or self.first_round
                prompt = "请输入上家出的牌(如:345或留空表示不出)"
                if must_play:
                    prompt += " (必须出牌): "
                else:
                    prompt += ": "
                    
                while True:
                    text = input(prompt).upper()
                    if not text:  # 不出
                        if must_play:
                            print("错误: 首回合不能不出或连续两个玩家不出后必须出牌！")
                            continue
                        else:
                            self.consecutive_passes += 1
                            self.other_played_cards_real = ""
                            break
                    # 验证牌型
                    if self.validate_card_pattern(text) and self.validate_and_update_card_library(text):
                        self.other_played_cards_real = text
                        self.consecutive_passes = 0
                        if self.first_round:
                            self.first_round = False
                        break
                    else:
                        print("错误: 牌型不符合规则或牌库中无此牌！")

                self.other_played_cards_env = [RealCard2EnvCard[c] for c in list(self.other_played_cards_real)]
                self.env.step(self.user_position, self.other_played_cards_env)
                self.play_order = 0
                print(f"[上家出牌]: {self.other_played_cards_real if self.other_played_cards_real else '不出'}")

        print("\n" + "="*50)
        print(f"{'农民' if self.env.winner == 'farmer' else '地主'}胜，本局结束!")
        print("="*50)
        self.env.reset()

    def validate_card_pattern(self, cards_str):
        """验证牌型是否符合规则"""
        if not cards_str:
            return True

        cards = list(cards_str)
        card_count = {}
        for card in cards:
            card_count[card] = card_count.get(card, 0) + 1


        if len(cards) == 1:
            return True


        if len(cards) == 2 and len(card_count) == 1:
            return True


        if len(cards) == 3 and len(card_count) == 1:
            return True


        if len(cards) == 4 and (list(card_count.values()).count(3) == 1 and list(card_count.values()).count(1) == 1):
            return True


        if len(cards) == 4 and len(card_count) == 1:
            return True


        if len(cards) >= 5 and len(card_count) == len(cards) and self.is_sequence(cards):
            return True


        if len(cards) >= 6 and len(cards) % 2 == 0 and all(
                count == 2 for count in card_count.values()) and self.is_sequence(sorted(card_count.keys())):
            return True

        return False

    def is_sequence(self, cards):
        """检查是否是连续的牌"""
        order = ['3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A', '2']
        try:
            card_indices = [order.index(c) for c in cards]
        except ValueError:
            return False
        return all(card_indices[i] + 1 == card_indices[i + 1] for i in range(len(card_indices) - 1))

if __name__ == '__main__':
    game = ConsoleDouDiZhu()
    game.run()

斗地主AI辅助工具 - 命令行版

游戏说明:
1. 输入手牌时使用字母表示: 3-9直接输入数字, 10=T, J=J, Q=Q, K=K, A=A, 2=2, 小王=X, 大王=D
2. 出牌时直接输入牌的组合，如 '345' 或 'TT'，或留空表示'不出'
3. 首回合不能不出，连续两个玩家不出后必须出牌

请输入你的手牌(如: 3456789TJQKA2XD): JJJJKKKKQQQQAAAA2
请输入三张底牌(如: 2XD): 222

请选择你的角色:
1. 地主上家
2. 地主
3. 地主下家
请输入数字(1-3): 2

当前牌库剩余:
3: 4张 | 4: 4张 | 5: 4张 | 6: 4张 | 7: 4张 | 8: 4张 | 9: 4张 | D: 1张 | T: 4张 | X: 1张 | 


C:\Users\Administrator\Desktop\Homework\AI\homework\ddz\DouZero_For_HappyDouDiZhu-master\douzero\evaluation\deep_agent.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  p


游戏开始！

当前手牌: JJJJQQQQKKKKAAAA2222

[AI正在思考...]

AI建议: JJQQKK (胜率: 100.0%)
[玩家出牌]: JJQQKK

当前手牌: JJQQKKAAAA2222
请输入下家出的牌(如:345或留空表示不出): 6666

当前牌库剩余:
3: 4张 | 4: 4张 | 5: 4张 | 7: 4张 | 8: 4张 | 9: 4张 | D: 1张 | T: 4张 | X: 1张 | 
[下家出牌]: 6666

当前手牌: JJQQKKAAAA2222
请输入上家出的牌(如:345或留空表示不出): 7777

当前牌库剩余:
3: 4张 | 4: 4张 | 5: 4张 | 8: 4张 | 9: 4张 | D: 1张 | T: 4张 | X: 1张 | 
[上家出牌]: 7777

当前手牌: JJQQKKAAAA2222

[AI正在思考...]

AI建议: 2222 (胜率: 82.85%)
[玩家出牌]: 2222

当前手牌: JJQQKKAAAA
请输入下家出的牌(如:345或留空表示不出): 
[下家出牌]: 不出

当前手牌: JJQQKKAAAA
请输入上家出的牌(如:345或留空表示不出) (必须出牌): 
错误: 首回合不能不出或连续两个玩家不出后必须出牌！


KeyboardInterrupt: Interrupted by user

In [1]:
# -*- coding: utf-8 -*-

from douzero.env.game import GameEnv
from douzero.evaluation.deep_agent import DeepAgent

EnvCard2RealCard = {3: '3', 4: '4', 5: '5', 6: '6', 7: '7',
                    8: '8', 9: '9', 10: 'T', 11: 'J', 12: 'Q',
                    13: 'K', 14: 'A', 17: '2', 20: 'X', 30: 'D'}

RealCard2EnvCard = {'3': 3, '4': 4, '5': 5, '6': 6, '7': 7,
                    '8': 8, '9': 9, 'T': 10, 'J': 11, 'Q': 12,
                    'K': 13, 'A': 14, '2': 17, 'X': 20, 'D': 30}

AllEnvCard = [3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7,
              8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12,
              12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 17, 17, 17, 17, 20, 30]

class ConsoleDouDiZhu:
    def __init__(self):
        # 模型路径
        self.card_play_model_path_dict = {
            'landlord': "baselines/douzero_WP/landlord.ckpt",
            'landlord_up': "baselines/douzero_WP/landlord_up.ckpt",
            'landlord_down': "baselines/douzero_WP/landlord_down.ckpt"
        }
        self.init_display()

    def init_display(self):
        print("="*50)
        print("斗地主AI辅助工具 - 命令行版")
        print("="*50)
        print("\n游戏说明:")
        print("1. 输入手牌时使用字母表示: 3-9直接输入数字, 10=T, J=J, Q=Q, K=K, A=A, 2=2, 小王=X, 大王=D")
        print("2. 出牌时直接输入牌的组合，如 '345' 或 'TT'，或留空表示'不出'")
        print("3. 首回合不能不出\n")

    def run(self):
        self.user_hand_cards_real = input("请输入你的手牌(如: 3456789TJQKA2XD): ").upper()
        self.three_landlord_cards_real = input("请输入三张底牌(如: 2XD): ").upper()
        
        print("\n请选择你的角色:")
        print("1. 地主上家")
        print("2. 地主")
        print("3. 地主下家")
        self.user_position_code = int(input("请输入数字(1-3): ")) - 1
        self.user_position = ['landlord_up', 'landlord', 'landlord_down'][self.user_position_code]
        
        if not self.initialize_card_library():
            print("牌库校验失败，请检查输入！")
            return
        
        self.process_after_input()

    def initialize_card_library(self):
        """初始化牌库，记录剩余牌数"""
        # 总牌库
        card_counts = {'3': 4, '4': 4, '5': 4, '6': 4, '7': 4, '8': 4, '9': 4, 'T': 4,
                      'J': 4, 'Q': 4, 'K': 4, 'A': 4, '2': 4, 'X': 1, 'D': 1}

        # 扣除玩家手牌
        for card in self.user_hand_cards_real:
            if card not in card_counts:
                print(f"错误: 无效的牌 '{card}'")
                return False
            card_counts[card] -= 1
            if card_counts[card] < 0:
                print(f"错误: 牌 '{card}' 的数量不足！")
                return False

        # 只有当玩家是地主时才扣除底牌
        if self.user_position == "landlord":
            for card in self.three_landlord_cards_real:
                if card not in card_counts:
                    print(f"错误: 无效的牌 '{card}'")
                    return False
                card_counts[card] -= 1
                if card_counts[card] < 0:
                    print(f"错误: 牌 '{card}' 的数量不足！")
                    return False

        self.card_library = card_counts
        self.update_card_library_display()
        return True

    def update_card_library_display(self):
        """更新牌库显示"""
        print("\n当前牌库剩余:")
        for card, count in sorted(self.card_library.items()):
            if count > 0:
                print(f"{card}: {count}张", end=" | ")
        print()

    def validate_and_update_card_library(self, played_cards):
        """验证并更新牌库"""
        if not played_cards:  # 不出牌
            return True

        temp_lib = self.card_library.copy()
        for card in played_cards:
            if card not in temp_lib:
                print(f"错误: 牌 '{card}' 不存在！")
                return False
            if temp_lib[card] <= 0:
                print(f"错误: 牌 '{card}' 已用完！")
                return False
            temp_lib[card] -= 1

        self.card_library = temp_lib
        self.update_card_library_display()
        return True

    def process_after_input(self):
        # 整副牌减去玩家手上的牌，就是其他人的手牌
        self.user_hand_cards_env = [RealCard2EnvCard[c] for c in list(self.user_hand_cards_real)]
        self.three_landlord_cards_env = [RealCard2EnvCard[c] for c in list(self.three_landlord_cards_real)]
        
        # 如果玩家是地主，将底牌加入手牌
        if self.user_position == "landlord":
            self.user_hand_cards_env.extend(self.three_landlord_cards_env)
            self.user_hand_cards_real += self.three_landlord_cards_real
        
        self.other_hand_cards = []
        for i in set(AllEnvCard):
            self.other_hand_cards.extend([i] * (AllEnvCard.count(i) - self.user_hand_cards_env.count(i)))

        # 修正手牌分配逻辑
        if self.user_position == "landlord":
            # 玩家是地主(20张)，其他两家各17张
            landlord_up_cards = self.other_hand_cards[:17]
            landlord_down_cards = self.other_hand_cards[17:]
        elif self.user_position == "landlord_up":
            # 玩家是地主上家(17张)，地主20张(17+3)，地主下家17张
            landlord_cards = self.other_hand_cards[:17] + self.three_landlord_cards_env
            landlord_down_cards = self.other_hand_cards[17:34]  # 取接下来的17张
        else:  # landlord_down
            # 玩家是地主下家(17张)，地主20张(17+3)，地主上家17张
            landlord_cards = self.other_hand_cards[:17] + self.three_landlord_cards_env
            landlord_up_cards = self.other_hand_cards[17:34]  # 取接下来的17张

        # 构建手牌数据字典
        self.card_play_data_list = {
            'three_landlord_cards': self.three_landlord_cards_env,
            'landlord_up': landlord_up_cards if self.user_position != "landlord_up" else self.user_hand_cards_env,
            'landlord': landlord_cards if self.user_position != "landlord" else self.user_hand_cards_env,
            'landlord_down': landlord_down_cards if self.user_position != "landlord_down" else self.user_hand_cards_env
        }

        # 校验手牌数量
        if len(self.card_play_data_list["three_landlord_cards"]) != 3:
            print("错误: 底牌必须是3张！")
            return

        # 修改手牌数量校验逻辑
        expected_landlord_up = 17 if self.user_position != "landlord_up" else len(self.user_hand_cards_env)
        expected_landlord_down = 17 if self.user_position != "landlord_down" else len(self.user_hand_cards_env)
        expected_landlord = 20 if self.user_position != "landlord" else len(self.user_hand_cards_env)
        
        if len(self.card_play_data_list["landlord_up"]) != expected_landlord_up or \
                len(self.card_play_data_list["landlord_down"]) != expected_landlord_down or \
                len(self.card_play_data_list["landlord"]) != expected_landlord:
            print(f"错误: 初始手牌数目有误 (地主上家:{len(self.card_play_data_list['landlord_up'])}张, " +
                  f"地主下家:{len(self.card_play_data_list['landlord_down'])}张, " +
                  f"地主:{len(self.card_play_data_list['landlord'])}张)")
            return

        # 得到出牌顺序
        self.play_order = 0 if self.user_position == "landlord" else 1 if self.user_position == "landlord_up" else 2

        # 创建一个代表玩家的AI
        ai_players = [0, 0]
        ai_players[0] = self.user_position
        ai_players[1] = DeepAgent(self.user_position, self.card_play_model_path_dict[self.user_position])

        self.env = GameEnv(ai_players)
        self.start()

    def start(self):
        self.env.card_play_init(self.card_play_data_list)
        print("\n游戏开始！")
        self.consecutive_passes = 0  # 初始化计数器
        self.first_round = True  # 初始化首回合标志
        self.last_played_position = None  # 新增：记录最后出牌的玩家
        self.must_play = False  # 新增：标记是否必须出牌
        
        while not self.env.game_over:
            print("\n" + "="*30)
            print(f"当前手牌: {''.join([EnvCard2RealCard[c] for c in sorted(self.env.info_sets[self.user_position].player_hand_cards)])}")
            
            if self.play_order == 0:
                # 玩家出牌 - 自动获取AI建议
                print("\n[AI正在思考...]")
                action_message = self.env.step(self.user_position)
                action_str = action_message["action"] if action_message["action"] else "不出"
                
                print(f"\nAI建议: {action_str} (胜率: {action_message['win_rate']})")
                
                if self.must_play and action_str == "不出":
                    print("错误: 你必须出牌，不能选择'不出'！")
                    continue
                
                if action_str == "不出":
                    self.consecutive_passes += 1
                    # 如果两个玩家都pass了，回到最后出牌的玩家
                    if self.consecutive_passes >= 2 and self.last_played_position is not None:
                        self.play_order = {'landlord':0, 'landlord_up':1, 'landlord_down':2}[self.last_played_position]
                        self.consecutive_passes = 0
                        self.must_play = True  # 必须出牌
                        print(f"\n[新回合] 由{self.last_played_position}开始新一轮出牌 (必须出牌)")
                        continue
                else:
                    self.consecutive_passes = 0
                    self.last_played_position = self.user_position  # 记录最后出牌的玩家
                    self.must_play = False  # 重置必须出牌标志
                
                self.first_round = False
                self.play_order = 1
                print(f"[玩家出牌]: {action_str}")

            elif self.play_order == 1:
                # 下家出牌 - 手动输入
                must_play = self.first_round or self.must_play  # 首回合或必须出牌时不能pass
                prompt = "请输入下家出的牌(如:345或留空表示不出)"
                if must_play:
                    prompt += " (必须出牌): "
                else:
                    prompt += ": "
                    
                while True:
                    text = input(prompt).upper()
                    if not text:  # 不出
                        if must_play:
                            print("错误: 必须出牌，不能选择'不出'！")
                            continue
                        else:
                            self.consecutive_passes += 1
                            # 如果两个玩家都pass了，回到最后出牌的玩家
                            if self.consecutive_passes >= 2 and self.last_played_position is not None:
                                self.play_order = {'landlord':0, 'landlord_up':1, 'landlord_down':2}[self.last_played_position]
                                self.consecutive_passes = 0
                                self.must_play = True  # 必须出牌
                                print(f"\n[新回合] 由{self.last_played_position}开始新一轮出牌 (必须出牌)")
                                break
                            self.other_played_cards_real = ""
                            break
                    # 验证牌型
                    if self.validate_card_pattern(text) and self.validate_and_update_card_library(text):
                        self.other_played_cards_real = text
                        self.consecutive_passes = 0
                        self.last_played_position = 'landlord_down' if self.user_position == 'landlord_up' else 'landlord_up'  # 记录最后出牌的玩家
                        self.must_play = False  # 重置必须出牌标志
                        if self.first_round:
                            self.first_round = False
                        break
                    else:
                        print("错误: 牌型不符合规则或牌库中无此牌！")

                if self.consecutive_passes >= 2:  # 已经处理了连续两个pass的情况
                    continue
                    
                self.other_played_cards_env = [RealCard2EnvCard[c] for c in list(self.other_played_cards_real)]
                self.env.step(self.user_position, self.other_played_cards_env)
                self.play_order = 2
                print(f"[下家出牌]: {self.other_played_cards_real if self.other_played_cards_real else '不出'}")

            elif self.play_order == 2:
                # 上家出牌 - 手动输入
                must_play = self.first_round or self.must_play  # 首回合或必须出牌时不能pass
                prompt = "请输入上家出的牌(如:345或留空表示不出)"
                if must_play:
                    prompt += " (必须出牌): "
                else:
                    prompt += ": "
                    
                while True:
                    text = input(prompt).upper()
                    if not text:  # 不出
                        if must_play:
                            print("错误: 必须出牌，不能选择'不出'！")
                            continue
                        else:
                            self.consecutive_passes += 1
                            # 如果两个玩家都pass了，回到最后出牌的玩家
                            if self.consecutive_passes >= 2 and self.last_played_position is not None:
                                self.play_order = {'landlord':0, 'landlord_up':1, 'landlord_down':2}[self.last_played_position]
                                self.consecutive_passes = 0
                                self.must_play = True  # 必须出牌
                                print(f"\n[新回合] 由{self.last_played_position}开始新一轮出牌 (必须出牌)")
                                break
                            self.other_played_cards_real = ""
                            break
                    # 验证牌型
                    if self.validate_card_pattern(text) and self.validate_and_update_card_library(text):
                        self.other_played_cards_real = text
                        self.consecutive_passes = 0
                        self.last_played_position = 'landlord_up' if self.user_position == 'landlord_down' else 'landlord'  # 记录最后出牌的玩家
                        self.must_play = False  # 重置必须出牌标志
                        if self.first_round:
                            self.first_round = False
                        break
                    else:
                        print("错误: 牌型不符合规则或牌库中无此牌！")

                if self.consecutive_passes >= 2:  # 已经处理了连续两个pass的情况
                    continue
                    
                self.other_played_cards_env = [RealCard2EnvCard[c] for c in list(self.other_played_cards_real)]
                self.env.step(self.user_position, self.other_played_cards_env)
                self.play_order = 0
                print(f"[上家出牌]: {self.other_played_cards_real if self.other_played_cards_real else '不出'}")

        print("\n" + "="*50)
        print(f"{'农民' if self.env.winner == 'farmer' else '地主'}胜，本局结束!")
        print("="*50)
        self.env.reset()

    def validate_card_pattern(self, cards_str):
        """验证牌型是否符合规则"""
        if not cards_str:
            return True

        cards = list(cards_str)
        card_count = {}
        for card in cards:
            card_count[card] = card_count.get(card, 0) + 1
            
        if len(cards) == 2 and set(cards) == {'X', 'D'}:
            return True

        if len(cards) == 1:
            return True


        if len(cards) == 2 and len(card_count) == 1:
            return True


        if len(cards) == 3 and len(card_count) == 1:
            return True


        if len(cards) == 4 and (list(card_count.values()).count(3) == 1 and list(card_count.values()).count(1) == 1):
            return True


        if len(cards) == 4 and len(card_count) == 1:
            return True


        if len(cards) >= 5 and len(card_count) == len(cards) and self.is_sequence(cards):
            return True


        if len(cards) >= 6 and len(cards) % 2 == 0 and all(
                count == 2 for count in card_count.values()) and self.is_sequence(sorted(card_count.keys())):
            return True
        

        return False

    def is_sequence(self, cards):
        """检查是否是连续的牌"""
        order = ['3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A', '2']
        try:
            card_indices = [order.index(c) for c in cards]
        except ValueError:
            return False
        return all(card_indices[i] + 1 == card_indices[i + 1] for i in range(len(card_indices) - 1))

if __name__ == '__main__':
    game = ConsoleDouDiZhu()
    game.run()

斗地主AI辅助工具 - 命令行版

游戏说明:
1. 输入手牌时使用字母表示: 3-9直接输入数字, 10=T, J=J, Q=Q, K=K, A=A, 2=2, 小王=X, 大王=D
2. 出牌时直接输入牌的组合，如 '345' 或 'TT'，或留空表示'不出'
3. 首回合不能不出



请输入你的手牌(如: 3456789TJQKA2XD):  33333333333333333
请输入三张底牌(如: 2XD):  222



请选择你的角色:
1. 地主上家
2. 地主
3. 地主下家


请输入数字(1-3):  1


错误: 牌 '3' 的数量不足！
牌库校验失败，请检查输入！
